
# HDA Flowsheet Simulation and Optimization


## Learning outcomes


- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a  flowsheet using Arcs
- Using the SequentialDecomposition tool to initialize a flowsheet with recycle
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing toluene and hydrogen to produce at least 370 TPY of benzene. As shown in the flowsheet, there are two flash tanks, F101 to separate out the non-condensibles and F102 to further separate the benzene-toluene mixture to improve the benzene purity.  Note that typically a distillation column is required to obtain high purity benzene but that is beyond the scope of this workshop. The non-condensibles separated out in F101 will be partially recycled back to M101 and the rest will be either purged or combusted for power generation.We will assume ideal gas for this flowsheet. The properties required for this module are available in the same directory:

- hda_ideal_VLE.py
- hda_reaction.py

The state variables chosen for the property package are **flows of component by phase, temperature and pressure**. The components considered are: **toluene, hydrogen, benzene and methane**. Therefore, every stream has 8 flow variables, 1 temperature and 1 pressure variable. 

![](HDA_flowsheet.png)




## Importing required pyomo and idaes components


To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- SolverFactory (to solve the problem)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)
- SequentialDecomposition (to initialize the flowsheet in a sequential mode)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/latest/


In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- StoichiometricReactor
- <span style="color:blue">**Flash**</span>
- Separator (splitter) 
- PressureChanger

In [2]:
from idaes.core import FlowsheetBlock

In [3]:
from idaes.models.unit_models import (PressureChanger,
                                      Mixer,
                                      Separator as Splitter,
                                      Heater,
                                      StoichiometricReactor)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, import the remaining unit models highlighted in blue above and run the cell using `Shift+Enter` after typing in the code. 
</div>


In [4]:
#Todo: import flash model from idaes.models.unit_models


In [5]:
#Todo: import flash model from idaes.models.unit_models
from idaes.models.unit_models import Flash

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom. 

In [6]:
from idaes.models.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom

# Import idaes logger to set output levels
import idaes.logger as idaeslog

## Importing required thermo and reaction package

The final set of imports are to import the thermo and reaction package for the HDA process. We have created a custom thermo package that assumes Ideal Gas with support for VLE. 

The reaction package here is very simple as we will be using only a StochiometricReactor and the reaction package consists of the stochiometric coefficients for the reaction and the parameter for the heat of reaction. 

Let us import the following modules and they are in the same directory as this jupyter notebook:
      <ul>
         <li>hda_ideal_VLE as thermo_props</li>
         <li>hda_reaction as reaction_props </li>
      </ul>
</div>

In [7]:
import sys
import os
commonpath = os.path.join(os.getcwd(), '../../common')
sys.path.append(commonpath)
import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props

## Constructing the Flowsheet

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block as we did in module 1. 

In [8]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike Module 1, where we only had a thermo property package, for this flowsheet we will also need to add a reaction property package. 

In [9]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        default={"property_package": m.fs.thermo_params})

## Adding Unit Models

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that, all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details (https://idaes-pse.readthedocs.io/en/latest/technical_specs/model_libraries/generic/unit_models/index.html). For example, the Mixer unit model here is given a `list` consisting of names to the three inlets. 

In [10]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]})

m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now add the StoichiometricReactor(assign the name R101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"reaction_package": m.fs.reaction_params </li>
         <li>"has_heat_of_reaction": True </li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>
</div>

In [11]:
#Todo: Add reactor with the specifications above


In [12]:
#Todo: Add reactor with the specifications above
m.fs.R101 = StoichiometricReactor(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

Let us now add the Flash(assign the name F101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>

In [13]:
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

Let us now add the Splitter(S101), PressureChanger(C101) and the second Flash(F102). 

In [14]:
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                               "ideal_separation": False,
                               "outlet_list": ["purge", "recycle"]})
    

m.fs.C101 = PressureChanger(default={
            "property_package": m.fs.thermo_params,
            "compressor": True,
            "thermodynamic_assumption": ThermodynamicAssumption.isothermal})
    
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

## Connecting Unit Models using Arcs

We have now added all the unit models we need to the flowsheet. However, we have not yet specifed how the units are to be connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer(M101) to the inlet of the heater(H101). 

In [15]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)


![](HDA_flowsheet.png) 

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, connect the H101 outlet to the R101 inlet using the cell above as a guide. 
</div>



In [16]:
#Todo: Connect the H101 outlet to R101 inlet


In [17]:
#Todo: Connect the H101 outlet to R101 inlet
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

We will now be connecting the rest of the flowsheet as shown below. Notice how the outlet names are different for the flash tanks F101 and F102 as they have a vapor and a liquid outlet. 

In [18]:
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
               destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [19]:
TransformationFactory("network.expand_arcs").apply_to(m)

## Adding expressions to compute purity and operating costs

In this section, we will add a few Expressions that allows us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html

For this flowsheet, we are interested in computing the purity of the product Benzene stream (i.e. the mole fraction) and the operating cost which is a sum of the cooling and heating cost. 

Let us first add an Expression to compute the mole fraction of benzene in the `vap_outlet` of F102 which is our product stream. Please note that the var flow_mol_phase_comp has the index - [time, phase, component]. As this is a steady-state flowsheet, the time index by default is 0. The valid phases are ["Liq", "Vap"]. Similarly the valid component list is ["benzene", "toluene", "hydrogen", "methane"].

In [20]:
m.fs.purity = Expression(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] /
        (m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
         + m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

Now, let us add an expression to compute the cooling cost assuming a cost of 0.212E-4 $/kW. Note that cooling utility is required for the reactor (R101) and the first flash (F101). 

In [21]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * (-m.fs.F101.heat_duty[0]) +
                                   0.212e-7 * (-m.fs.R101.heat_duty[0]))


Now, let us add an expression to compute the heating cost assuming the utility cost as follows:
      <ul>
         <li>2.2E-4 dollars/kW for H101</li>
         <li>1.9E-4 dollars/kW for F102</li>
      </ul>
Note that the heat duty is in units of watt (J/s). 

In [22]:
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                   1.9e-7 * m.fs.F102.heat_duty[0])

Let us now add an expression to compute the total operating cost per year which is basically the sum of the cooling and heating cost we defined above. 

In [23]:
m.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                           (m.fs.heating_cost +
                                            m.fs.cooling_cost)))

## Fixing feed conditions

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. 

In [24]:
print(degrees_of_freedom(m))

29


In [25]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 29

We will now be fixing the toluene feed stream to the conditions shown in the flowsheet above. Please note that though this is a pure toluene feed, the remaining components are still assigned a very small non-zero value to help with convergence and initializing. 

In [26]:
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)


Similarly, let us fix the hydrogen feed to the following conditions in the next cell:
      <ul>
         <li>F<sub>H2</sub> = 0.30 mol/s</li>
         <li>F<sub>CH4</sub> = 0.02 mol/s</li>
         <li>Remaining components = 1e-5 mol/s</li>
         <li>T = 303.2 K</li>
         <li>P = 350000 Pa</li>
      </ul>



In [27]:
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

## Fixing unit model specifications

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us set set the H101 outlet temperature to 600 K. 

In [28]:
m.fs.H101.outlet.temperature.fix(600)

For the StoichiometricReactor, we have to define the conversion in terms of toluene. This requires us to create a new variable for specifying the conversion and adding a Constraint that defines the conversion with respect to toluene. The second degree of freedom for the reactor is to define the heat duty. In this case, let us assume the reactor to be adiabatic i.e. Q = 0. 

In [29]:
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.
    flow_mol_phase_comp[0, "Vap", "toluene"] ==
    (m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"] -
     m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

m.fs.R101.conversion.fix(0.75)
m.fs.R101.heat_duty.fix(0)

The Flash conditions for F101 can be set as follows. 

In [30]:
m.fs.F101.vap_outlet.temperature.fix(325.0)
m.fs.F101.deltaP.fix(0)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Set the conditions for Flash F102 to the following conditions:
      <ul>
         <li>T = 375 K</li>
         <li>deltaP = -200000</li>
      </ul>

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [31]:
#Todo: Set conditions for Flash F102


In [32]:
m.fs.F102.vap_outlet.temperature.fix(375)
m.fs.F102.deltaP.fix(-200000)

Let us fix the purge split fraction to 20% and the outlet pressure of the compressor is set to 350000 Pa. 

In [33]:
m.fs.S101.split_fraction[0, "purge"].fix(0.2)
m.fs.C101.outlet.pressure.fix(350000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined all the feed conditions and the inputs required for the unit models. The system should now have 0 degrees of freedom i.e. should be a square problem. Please check that the degrees of freedom is 0. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [34]:
#Todo: print the degrees of freedom


In [35]:
print(degrees_of_freedom(m))

0


In [36]:
# Check the degrees of freedom
assert degrees_of_freedom(m) == 0

## Initialization


This section will demonstrate how to use the built-in sequential decomposition tool to initialize our flowsheet.

![](HDA_flowsheet.png) 


Let us first create an object for the SequentialDecomposition and specify our options for this. 

In [37]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 3

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

Which is the tear stream? Display tear set and order

In [38]:
for o in heuristic_tear_set:
    print(o.name)

fs.s03


What sequence did the SD tool determine to solve this flowsheet with the least number of tears? 

In [39]:
for o in order:
    print(o[0].name)

fs.H101
fs.R101
fs.F101
fs.S101
fs.C101
fs.M101


 

![](HDA_tear_stream.png) 


The SequentialDecomposition tool has determined that the tear stream is the mixer outlet. We will need to provide a reasonable guess for this.

In [40]:
tear_guesses = {
        "flow_mol_phase_comp": {
                (0, "Vap", "benzene"): 1e-5,
                (0, "Vap", "toluene"): 1e-5,
                (0, "Vap", "hydrogen"): 0.30,
                (0, "Vap", "methane"): 0.02,
                (0, "Liq", "benzene"): 1e-5,
                (0, "Liq", "toluene"): 0.30,
                (0, "Liq", "hydrogen"): 1e-5,
                (0, "Liq", "methane"): 1e-5},
        "temperature": {0: 303},
        "pressure": {0: 350000}}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

Next, we need to tell the tool how to initialize a particular unit. We will be writing a python function which takes in a "unit" and calls the initialize method on that unit. 

In [41]:
def function(unit):
        unit.initialize(outlvl=idaeslog.DEBUG)

We are now ready to initialize our flowsheet in a sequential mode. Note that we specifically set the iteration limit to be 5 as we are trying to use this tool only to get a good set of initial values such that IPOPT can then take over and solve this flowsheet for us. 

In [42]:
seq.run(m, function)

2022-08-04 07:20:54 [INFO] idaes.init.fs.H101.control_volume.properties_in: Initialization Step 1 - Dew and bubble points calculation completed.
2022-08-04 07:20:54 [INFO] idaes.init.fs.H101.control_volume.properties_in: Initialization Step 2 - Equilibrium temperature  calculation completed.
2022-08-04 07:20:54 [DEBUG] idaes.solve.fs.H101.control_volume.properties_in: Ipopt 3.13.2: nlp_scaling_method=gradient-based
2022-08-04 07:20:54 [DEBUG] idaes.solve.fs.H101.control_volume.properties_in: tol=1e-06
2022-08-04 07:20:54 [DEBUG] idaes.solve.fs.H101.control_volume.properties_in: ******************************************************************************
2022-08-04 07:20:54 [DEBUG] idaes.solve.fs.H101.control_volume.properties_in: This program contains Ipopt, a library for large-scale nonlinear optimization.
2022-08-04 07:20:54 [DEBUG] idaes.solve.fs.H101.control_volume.properties_in: Ipopt is released as open source code under the Eclipse Public License (EPL).
2022-08-04 07:20:54 [DE

2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: For more information visit http://projects.coin-or.org/Ipopt
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: This version of Ipopt was compiled from source code available at
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: https://github.com/IDAES/Ipopt as part of the Institute for the Design of
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: This version of Ipopt was compiled using HSL, a collection of Fortran codes
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101.control_volume.properties_out: for large-scale scient

2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: Ipopt is released as open source code under the Eclipse Public License (EPL).
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: For more information visit http://projects.coin-or.org/Ipopt
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: This version of Ipopt was compiled from source code available at
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: https://github.com/IDAES/Ipopt as part of the Institute for the Design of
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: This version of Ipopt was compiled using HSL, a collection of Fortran codes
2022-08-04 07:20:55 [DEBUG] idaes.solve.fs.H101: for large-scale scientific computation.  All technical papers, sales and
2022-08-04 07:20:55 [DEBUG] idae

2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 38r 0.0000000e+00 4.29e+03 4.56e+04  -3.9 9.63e+03  -5.9 1.00e+00 6.91e-04f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 39r 0.0000000e+00 4.25e+03 4.71e+04  -3.9 2.91e+05    -  6.20e-02 8.82e-03f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 40r 0.0000000e+00 4.20e+03 6.91e+04  -3.9 2.00e+05    -  8.94e-01 1.13e-02f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 41r 0.0000000e+00 4.11e+03 6.67e+04  -3.9 1.49e+05    -  1.38e-01 2.22e-02f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 42r 0.0000000e+00 3.20e+03 4.92e+04  -3.9 1.45e+05    -  1.53e-01 2.23e-01f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 43r 0.0000000e+00 1.51e+03 9.84e+03  -3.9 1.13e+05    -  1.00e+00 5.35e-01f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 44r 0.0000000e+00 9.72e+02 4.26e+04  -3.9 2.82e+04   

2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 96r 0.0000000e+00 4.94e-03 1.79e+01  -3.9 1.43e-01  -0.5 1.00e+00 1.00e+00f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 97r 0.0000000e+00 6.07e+01 1.30e+06  -3.9 2.64e+04    -  1.00e+00 1.16e-03f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 98r 0.0000000e+00 7.25e+01 5.06e+04  -3.9 2.57e+03    -  1.00e+00 3.50e-01f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 99r 0.0000000e+00 6.75e+01 1.65e+06  -3.9 5.60e+02    -  1.48e-01 6.64e-01h  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 100r 0.0000000e+00 5.42e+01 2.75e+06  -3.9 2.27e+02    -  1.00e+00 2.18e-01f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 101r 0.0000000e+00 5.15e+01 5.37e+06  -3.9 1.72e+02    -  1.67e-01 1.58e-01f  1
2022-08-04 07:20:56 [DEBUG] idaes.solve.fs.H101: 102r 0.0000000e+00 3.28e+02 1.83e+08  -3.9 1.61e+02

...

2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: Ipopt 3.13.2: nlp_scaling_method=gradient-based
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: tol=1e-06
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: ******************************************************************************
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: This program contains Ipopt, a library for large-scale nonlinear optimization.
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: Ipopt is released as open source code under the Eclipse Public License (EPL).
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: For more information visit http://projects.coin-or.org/Ipopt
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: This version of Ipopt was compiled from source code available at
2022-08-04 07:21:21 [DEBUG] idaes.solve.fs.M101.mixed_state: https://github.com/IDAES/Ipopt as part of the Institute for the Design of
2022-08-04 07:21:2

2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: This is Ipopt version 3.13.2, running with linear solver ma27.
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: Number of nonzeros in equality constraint Jacobian...:      117
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: Number of nonzeros in inequality constraint Jacobian.:        0
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: Number of nonzeros in Lagrangian Hessian.............:       63
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: Total number of variables............................:       53
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: variables with only lower bounds:        0
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: variables with lower and upper bounds:       10
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: variables with only upper bounds:        0
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs.M101: Total number of equality constraints.................:       53
2022-08-04 07:21:22 [DEBUG] idaes.solve.fs

2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: Total number of inequality constraints...............:        0
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: inequality constraints with only lower bounds:        0
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: inequality constraints with lower and upper bounds:        0
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: inequality constraints with only upper bounds:        0
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: 0  0.0000000e+00 2.27e-13 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_in: Number of Iterations....: 0
2022-08-04 07:21:23 [DEBUG] id

2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: (scaled)                 (unscaled)
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: Dual infeasibility......:   0.0000000000000000e+00    0.0000000000000000e+00
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: Constraint violation....:   4.5474735088646412e-13    4.5474735088646412e-13
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: Overall NLP error.......:   4.5474735088646412e-13    4.5474735088646412e-13
2022-08-04 07:21:23 [DEBUG] idaes.solve.fs.F102.control_volume.properties_out: Number of objective functi

2022-08-04 07:21:24 [DEBUG] idaes.solve.fs.F102: Number of inequality constraint Jacobian evaluations = 0
2022-08-04 07:21:24 [DEBUG] idaes.solve.fs.F102: Number of Lagrangian Hessian evaluations             = 7
2022-08-04 07:21:24 [DEBUG] idaes.solve.fs.F102: Total CPU secs in IPOPT (w/o function evaluations)   =      0.011
2022-08-04 07:21:24 [DEBUG] idaes.solve.fs.F102: Total CPU secs in NLP function evaluations           =      0.001
2022-08-04 07:21:24 [DEBUG] idaes.solve.fs.F102: EXIT: Optimal Solution Found.
2022-08-04 07:21:24 [INFO] idaes.init.fs.F102: Initialization Step 2 optimal - Optimal Solution Found.
2022-08-04 07:21:24 [INFO] idaes.init.fs.F102.control_volume.properties_in: State Released.
2022-08-04 07:21:24 [INFO] idaes.init.fs.F102: Initialization Complete: optimal - Optimal Solution Found


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now initialized the flowsheet. Let us run the flowsheet in a simulation mode to look at the results. To do this, complete the last line of code where we pass the model to the solver. You will need to type the following:
    
results = solver.solve(m, tee=True)

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [43]:
# Create the solver object


# Solve the model


In [44]:
# Create the solver object
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6, 'max_iter': 5000}

# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        

In [45]:
# Check solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

## Analyze the results of the square problem


What is the total operating cost? 

In [46]:
print('operating cost = $', value(m.fs.operating_cost))

operating cost = $ 419122.33876779437


In [47]:
import pytest
assert value(m.fs.operating_cost) == pytest.approx(419122.3387, abs=1e-3)

For this operating cost, what is the amount of benzene we are able to produce and what purity we are able to achieve? 

In [48]:
m.fs.F102.report()

print()
print('benzene purity = ', value(m.fs.purity))



Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value       : Units  : Fixed : Bounds
          Heat Duty :      7352.5 :   watt : False : (None, None)
    Pressure Change : -2.0000e+05 : pascal :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')   mole / second    0.20460   1.0000e-08      0.062620  
    flow_mol_phase_comp ('Liq', 'toluene')   mole / second   0.062520   1.0000e-08      0.032257  
    flow_mol_phase_comp ('Liq', 'methane')   mole / second 2.6712e-07   1.0000e-08    9.4877e-08  
    flow_mol_phase_comp ('Liq', 'hydrogen')  mole / second 2.6712e-07   1.0000e-08    9.4877e-08  


In [49]:
assert value(m.fs.purity) == pytest.approx(0.82429, abs=1e-3)

assert value(m.fs.F102.heat_duty[0]) == pytest.approx(7352.4828, abs=1e-3)
assert value(m.fs.F102.vap_outlet.pressure[0]) == pytest.approx(1.5000e+05, abs=1e-3)

Next, let's look at how much benzene we are loosing with the light gases out of F101. IDAES has tools for creating stream tables based on the `Arcs` and/or `Ports` in a flowsheet. Let us create and print a simple stream table showing the stream leaving the reactor and the vapor stream from F101.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
How much benzene are we loosing in the F101 vapor outlet stream?
</div>


In [50]:
from idaes.core.util.tables import create_stream_table_dataframe, stream_table_dataframe_to_string

st = create_stream_table_dataframe({"Reactor": m.fs.s05, "Light Gases": m.fs.s06})
print(stream_table_dataframe_to_string(st))

                                            Units        Reactor   Light Gases
flow_mol_phase_comp ('Liq', 'benzene')   mole / second 1.2993e-07  1.0000e-08 
flow_mol_phase_comp ('Liq', 'toluene')   mole / second 8.4147e-07  1.0000e-08 
flow_mol_phase_comp ('Liq', 'methane')   mole / second 1.0000e-12  1.0000e-08 
flow_mol_phase_comp ('Liq', 'hydrogen')  mole / second 1.0000e-12  1.0000e-08 
flow_mol_phase_comp ('Vap', 'benzene')   mole / second    0.35374     0.14915 
flow_mol_phase_comp ('Vap', 'toluene')   mole / second   0.078129    0.015610 
flow_mol_phase_comp ('Vap', 'methane')   mole / second     1.2721      1.2721 
flow_mol_phase_comp ('Vap', 'hydrogen')  mole / second    0.32821     0.32821 
temperature                                     kelvin     771.85      325.00 
pressure                                        pascal 3.5000e+05  3.5000e+05 


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
You can querry additional variables here if you like. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>


## Optimization


We saw from the results above that the total operating cost for the base case was $419,122 per year. We are producing 0.142 mol/s of benzene at a purity of 82\%. However, we are losing around 42\% of benzene in F101 vapor outlet stream. 

Let us try to minimize this cost such that:
- we are producing at least 0.15 mol/s of benzene in F102 vapor outlet i.e. our product stream
- purity of benzene i.e. the mole fraction of benzene in F102 vapor outlet is at least 80%
- restricting the benzene loss in F101 vapor outlet to less than 20%

For this problem, our decision variables are as follows:
- H101 outlet temperature
- R101 cooling duty provided
- F101 outlet temperature
- F102 outlet temperature
- F102 deltaP in the flash tank


Let us declare our objective function for this problem. 

In [51]:
m.fs.objective = Objective(expr=m.fs.operating_cost)

Now, we need to unfix the decision variables as we had solved a square problem (degrees of freedom = 0) until now. 

In [52]:
m.fs.H101.outlet.temperature.unfix()
m.fs.R101.heat_duty.unfix()
m.fs.F101.vap_outlet.temperature.unfix()
m.fs.F102.vap_outlet.temperature.unfix()

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now unfix the remaining variable which is F102 pressure drop (F102.deltaP) 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [53]:
#Todo: Unfix deltaP for F102


In [54]:
#Todo: Unfix deltaP for F102
m.fs.F102.deltaP.unfix()

In [55]:
assert degrees_of_freedom(m) == 5

Next, we need to set bounds on these decision variables to values shown below:

 - H101 outlet temperature [500, 600] K
 - R101 outlet temperature [600, 800] K
 - F101 outlet temperature [298, 450] K
 - F102 outlet temperature [298, 450] K
 - F102 outlet pressure [105000, 110000] Pa

Let us first set the variable bound for the H101 outlet temperature as shown below:

In [56]:
m.fs.H101.outlet.temperature[0].setlb(500)
m.fs.H101.outlet.temperature[0].setub(600)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, set the variable bound for the R101 outlet temperature.

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [57]:
#Todo: Set the bounds for reactor outlet temperature


In [58]:
#Todo: Set the bounds for reactor outlet temperature
m.fs.R101.outlet.temperature[0].setlb(600)
m.fs.R101.outlet.temperature[0].setub(800)

Let us fix the bounds for the rest of the decision variables. 

In [59]:
m.fs.F101.vap_outlet.temperature[0].setlb(298.0)
m.fs.F101.vap_outlet.temperature[0].setub(450.0)
m.fs.F102.vap_outlet.temperature[0].setlb(298.0)
m.fs.F102.vap_outlet.temperature[0].setub(450.0)
m.fs.F102.vap_outlet.pressure[0].setlb(105000)
m.fs.F102.vap_outlet.pressure[0].setub(110000)

Now, the only things left to define are our constraints on overhead loss in F101, product flow rate and purity in F102. Let us first look at defining a constraint for the overhead loss in F101 where we are restricting the benzene leaving the vapor stream to less than 20 \% of the benzene available in the reactor outlet. 

In [60]:
m.fs.overhead_loss = Constraint(
        expr=m.fs.F101.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] <=
        0.20 * m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "benzene"])

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, add the constraint such that we are producing at least 0.15 mol/s of benzene in the product stream which is the vapor outlet of F102. Let us name this constraint as m.fs.product_flow. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [61]:
#Todo: Add minimum product flow constraint


In [62]:
#Todo: Add minimum product flow constraint
m.fs.product_flow = Constraint(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] >=
        0.15)

Let us add the final constraint on product purity or the mole fraction of benzene in the product stream such that it is at least greater than 80%. 

In [63]:
m.fs.product_purity = Constraint(expr=m.fs.purity >= 0.80)


We have now defined the optimization problem and we are now ready to solve this problem. 




In [64]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        

In [65]:
# Check for solver solve status
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

## Optimization Results

Display the results and product specifications

In [66]:
print('operating cost = $', value(m.fs.operating_cost))

print()
print('Product flow rate and purity in F102')

m.fs.F102.report()

print()
print('benzene purity = ', value(m.fs.purity))

print()
print('Overhead loss in F101')
m.fs.F101.report()

operating cost = $ 312786.3383410268

Product flow rate and purity in F102

Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value       : Units  : Fixed : Bounds
          Heat Duty :      8377.0 :   watt : False : (None, None)
    Pressure Change : -2.4500e+05 : pascal : False : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                                Units         Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')   mole / second    0.21743   1.0000e-08      0.067425  
    flow_mol_phase_comp ('Liq', 'toluene')   mole / second   0.070695   1.0000e-08      0.037507  
    flow_mol_phase_comp ('Liq', 'methane')   mole / second 2.8812e-07   1.0000e-08    1.0493e-07  
    flow_mol_phase_comp 

In [67]:
assert value(m.fs.operating_cost) == pytest.approx(312786.338, abs=1e-3)
assert value(m.fs.purity) == pytest.approx(0.818827, abs=1e-3)

Display optimal values for the decision variables

In [68]:
print('Optimal Values')
print()

print('H101 outlet temperature = ', value(m.fs.H101.outlet.temperature[0]), 'K')

print()
print('R101 outlet temperature = ', value(m.fs.R101.outlet.temperature[0]), 'K')

print()
print('F101 outlet temperature = ', value(m.fs.F101.vap_outlet.temperature[0]), 'K')

print()
print('F102 outlet temperature = ', value(m.fs.F102.vap_outlet.temperature[0]), 'K')
print('F102 outlet pressure = ', value(m.fs.F102.vap_outlet.pressure[0]), 'Pa')

Optimal Values

H101 outlet temperature =  500.0 K

R101 outlet temperature =  696.1117584980856 K

F101 outlet temperature =  301.87847605692826 K

F102 outlet temperature =  362.93476830548985 K
F102 outlet pressure =  105000.0 Pa


In [69]:
assert value(m.fs.H101.outlet.temperature[0]) == pytest.approx(500, abs=1e-3)
assert value(m.fs.R101.outlet.temperature[0]) == pytest.approx(696.112, abs=1e-3)
assert value(m.fs.F101.vap_outlet.temperature[0]) == pytest.approx(301.878, abs=1e-3)
assert value(m.fs.F102.vap_outlet.temperature[0]) == pytest.approx(362.935, abs=1e-3)
assert value(m.fs.F102.vap_outlet.pressure[0]) == pytest.approx(105000, abs=1e-2)